In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_openml
from PIL import Image
import io

import ipywidgets as widgets
from IPython.display import display

In [ ]:
print("Đang tải MNIST...")
mnist = fetch_openml('mnist_784', version=1)

X = mnist.data / 255.0
y = mnist.target.astype(int)

# 👉 Tăng dữ liệu huấn luyện
X_train = X[:30000]
y_train = y[:30000]

# 👉 Tăng số láng giềng
knn = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn.fit(X_train, y_train)

print("Huấn luyện xong!")


Đang tải MNIST...
Huấn luyện xong!


In [ ]:
#Tải bộ dữ liệu MNIST
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)
#KNN yêu cầu dữ liệu dạng vector, nên ta flatten ảnh 28×28 → 784
# Chuyển sang dạng float và chuẩn hóa về [0,1]
X_train = X_train.reshape(60000, 784).astype("float32") / 255.0
X_test  = X_test.reshape(10000, 784).astype("float32") / 255.0

print("Train:", X_train.shape)
print("Test :", X_test.shape)

#Huấn Luyện Mô Hình
from sklearn.neighbors import KNeighborsClassifier

# Tạo mô hình KNN với K = 3
knn = KNeighborsClassifier(n_neighbors=3)

# Huấn luyện
print("Đang huấn luyện KNN ...")
knn.fit(X_train, y_train)

print("Huấn luyện xong!")

#Dự đoán và đánh giá
from sklearn.metrics import accuracy_score, classification_report

y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Độ chính xác KNN:", acc)

print("\nBáo cáo phân loại:\n")
print(classification_report(y_test, y_pred))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Train: (60000, 28, 28) (60000,)
Test : (10000, 28, 28) (10000,)
Train: (60000, 784)
Test : (10000, 784)
Đang huấn luyện KNN ...
Huấn luyện xong!
Độ chính xác KNN: 0.9705

Báo cáo phân loại:

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.96      1.00      0.98      1135
           2       0.98      0.97      0.97      1032
           3       0.96      0.97      0.96      1010
           4       0.98      0.97      0.97       982
           5       0.97      0.96      0.96       892
           6       0.98      0.99      0.98       958
           7       0.96      0.96      0.96      1028
           8       0.99      0.94      0.96       974
           9       0.96      0.96      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import joblib

from tensorflow.keras.datasets import mnist

#  tải MNIST bằng Keras
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Chuyển 28x28 → vector 784 để dùng KNN
X_train = X_train.reshape(-1, 28*28) / 255.0
X_test  = X_test.reshape(-1, 28*28) / 255.0

print("Train:", X_train.shape)
print("Test:", X_test.shape)

X = X_train
y = y_train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Train model
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

# Lưu mô hình
joblib.dump(model, "mnist_knn.pkl")
print("Đã lưu mô hình mnist_knn.pkl")


Train: (60000, 784)
Test: (10000, 784)
Đã lưu mô hình mnist_knn.pkl


In [ ]:
# Load mô hình đã huấn luyện
import joblib
joblib.dump(knn, "knn_mnist_model.pkl")


['knn_mnist_model.pkl']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2

# ===== TIÊU ĐỀ =====
title = widgets.HTML(
    "<h2 style='color:#2c3e50; text-align:center;'>🖊️ Nhận diện chữ số viết tay</h2>"
)

subtitle = widgets.HTML(
    "<p style='text-align:center; color:gray;'>Upload ảnh chữ số (0–9) để hệ thống dự đoán</p>"
)

# ===== UPLOAD =====
upload = widgets.FileUpload(
    accept='.png,.jpg,.jpeg',
    multiple=False,
    description= 'Chọn ảnh tải lên',
    style={'button_color': '#3498db'}
)

# ===== NÚT DỰ ĐOÁN =====
btn_predict = widgets.Button(
    description=' dự đoán',
    button_style='success'
)

# ===== KẾT QUẢ =====
output = widgets.Output()

# ===== HÀM TIỀN XỬ LÝ (đã sửa đúng cho bạn) =====
def preprocess(img):
    img = np.array(img)

    # 1 Blur nhẹ để giảm nhiễu
    img = cv2.GaussianBlur(img, (5,5), 0)

    # 2 Adaptive threshold (ổn định cho ảnh chụp)
    img = cv2.adaptiveThreshold(
        img, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        11, 2
    )

    #  Lấy vùng chữ số
    coords = cv2.findNonZero(img)
    if coords is None:
        return np.zeros((28,28), dtype=np.uint8)

    x, y, w, h = cv2.boundingRect(coords)
    digit = img[y:y+h, x:x+w]

    #  Resize GIỮ TỈ LỆ về 20px
    size = 20
    h, w = digit.shape
    if h > w:
        digit = cv2.resize(digit, (int(size*w/h), size))
    else:
        digit = cv2.resize(digit, (size, int(size*h/w)))

       #  Padding về 28x28
    canvas = np.zeros((28,28), dtype=np.uint8)
    y0 = (28 - digit.shape[0]) // 2
    x0 = (28 - digit.shape[1]) // 2
    canvas[y0:y0+digit.shape[0], x0:x0+digit.shape[1]] = digit

    #  DILATE NHẸ để nối nét
    kernel = np.ones((2,2), np.uint8)
    canvas = cv2.dilate(canvas, kernel, iterations=1)

    return canvas



# ===== XỬ LÝ DỰ ĐOÁN =====
def on_predict(b):
    clear_output(wait=True)
    display(title, subtitle, controls)

    if len(upload.value) == 0:
        print("⚠️ Vui lòng chọn ảnh trước")
        return

    for name, file in upload.value.items():
        img = Image.open(io.BytesIO(file['content'])).convert("L")
        processed = preprocess(img)

        X = processed.reshape(1, -1) / 255.0
        pred = knn.predict(X)[0]

        with output:
            print(f"✅ Kết quả dự đoán: {pred}")

            fig, ax = plt.subplots(1,2, figsize=(7,3))
            ax[0].set_title("Ảnh gốc")
            ax[0].imshow(img, cmap='gray')
            ax[0].axis('off')

            ax[1].set_title("Ảnh sau xử lý")
            ax[1].imshow(processed, cmap='gray')
            ax[1].axis('off')

            plt.show()

# ===== GẮN SỰ KIỆN =====
btn_predict.on_click(on_predict)

controls = widgets.VBox([
    upload,
    btn_predict,
    output
])

display(title, subtitle, controls)


HTML(value="<h2 style='color:#2c3e50; text-align:center;'>🖊️ Nhận diện chữ số viết tay</h2>")

HTML(value="<p style='text-align:center; color:gray;'>Upload ảnh chữ số (0–9) để hệ thống dự đoán</p>")